# Created a merged corresponding authors dataset

Identifies corresponding authors from PubMed Central author list if available. Otherwise, use PubMed author list.

In [1]:
import pandas

In [2]:
pubmed_df = pandas.read_csv("data/pubmed/articles.tsv.xz", sep="\t")
pubmed_df.head(2)

,pmid,pmcid,doi,journal,publication_date,title
0,9520496,NaN,10.1093/bioinformatics/14.1.2,Bioinformatics,1998,The Merck Gene Index browser: an extensible da...
1,9520497,NaN,10.1093/bioinformatics/14.1.14,Bioinformatics,1998,Algorithms and software for support of gene id...


In [3]:
pubmed_author_df = pandas.read_csv("data/pubmed/authors.tsv.xz", sep="\t")
pubmed_author_df.head(2)

,pmid,position,fore_name,last_name,reverse_position
0,9520496,1,B A,Eckman,7
1,9520496,2,J S,Aaronson,6


In [4]:
pmc_author_df = pandas.read_csv("data/pmc/authors.tsv.xz", sep="\t")
pmc_author_df.head(2)

,pmcid,position,fore_name,last_name,corresponding,reverse_position
0,PMC100321,1,Alexander E,Pozhitkov,1,2
1,PMC100321,2,Diethard,Tautz,0,1


In [5]:
def get_corresponding(df):
    if df.corresponding.any():
        df = df.query("corresponding == 1")
        df['use_last'] = 0
    else:
        df = df.query("reverse_position == 1")
        df['use_last'] = 1
    return df.assign(n_corresponding=len(df))

pmc_corresp_df = (
    pubmed_df[['pmid', 'pmcid']]
    .merge(pmc_author_df)
    .assign(source="pmc")
    .groupby('pmid')
    .apply(get_corresponding)
)
pmc_corresp_df.head(2)

pmid     pmcid  position   fore_name last_name  corresponding  \
pmid                                                                            
11001586 0  11001586  PMC29061         1    Jeremy S   Edwards              1   
         1  11001586  PMC29061         2  Bernhard O   Palsson              1   

            reverse_position source  use_last  n_corresponding  
pmid                                                            
11001586 0                 2    pmc         0                2  
         1                 1    pmc         0                2

In [6]:
pubmed_corresp_df = (
    pubmed_author_df
    .query("pmid not in @pmc_corresp_df.pmid")
    .assign(corresponding=None, source="pubmed")
    .groupby('pmid')
    .apply(get_corresponding)
)
pubmed_corresp_df.head(2)

,,pmid,position,fore_name,last_name,reverse_position,corresponding,source,use_last,n_corresponding
pmid,,,,,,,,,,
9520496,6,9520496,7,R A,Blevins,1,None,pubmed,1,1
9520497,12,9520497,6,P A,Pevzner,1,None,pubmed,1,1


In [7]:
corresp_df = pandas.concat([pmc_corresp_df, pubmed_corresp_df], sort=False).reset_index(drop=True)
fore_df = pandas.read_csv("data/names/fore-names.tsv.xz", sep="\t")
last_df = pandas.read_csv("data/names/last-names.tsv.xz", sep="\t")
corresp_df = (
    corresp_df
    .merge(fore_df[["fore_name", "fore_name_simple"]], how="left")
    .merge(last_df[["last_name", "last_name_simple"]], how="left")
)
corresp_df.head(2)

,pmid,pmcid,position,fore_name,last_name,corresponding,reverse_position,source,use_last,n_corresponding,fore_name_simple,last_name_simple
0,11001586,PMC29061,1,Jeremy S,Edwards,1,2,pmc,0,2,jeremy,edwards
1,11001586,PMC29061,2,Bernhard O,Palsson,1,1,pmc,0,2,bernhard,palsson


In [8]:
source_df = (
    pubmed_df
    .merge(corresp_df[['pmid', 'source', 'use_last']].drop_duplicates())
)
source_df.head(2)

,pmid,pmcid,doi,journal,publication_date,title,source,use_last
0,9520496,NaN,10.1093/bioinformatics/14.1.2,Bioinformatics,1998,The Merck Gene Index browser: an extensible da...,pubmed,1
1,9520497,NaN,10.1093/bioinformatics/14.1.14,Bioinformatics,1998,Algorithms and software for support of gene id...,pubmed,1


In [9]:
# use_last prevalence
pandas.crosstab(source_df.source, source_df.use_last, margins=True)

use_last,0,1,All
source,,,
pmc,20576,120,20696
pubmed,0,9053,9053
All,20576,9173,29749


In [10]:
# number of articles by authorship source by journal
pandas.crosstab(source_df.journal, source_df.source, margins=True)

source,pmc,pubmed,All
journal,,,
BMC Bioinformatics,9389,19,9408
Bioinformatics,4816,9016,13832
PLoS Comput Biol,6491,18,6509
All,20696,9053,29749


In [11]:
# number of articles by authorship source by year
pandas.crosstab(source_df.publication_date.str.slice(0, 4), source_df.source, margins=True)

source,pmc,pubmed,All
publication_date,,,
1998,0,127,127
1999,0,121,121
2000,1,168,169
2001,9,229,238
2002,39,347,386
2003,66,514,580
2004,208,893,1101
2005,422,690,1112
2006,790,610,1400


In [12]:
corresp_df.to_csv('data/names/corresponding-authors.tsv.xz', sep='\t', index=False)